In [12]:
# DEPENDENCIES

import pysam
import pandas as pd
import pstats
from tqdm import tqdm as tqdm
import cProfile
import numpy as np

# GLOBAL VARS
FC30_DMGOTH_MAX_AS_PRIMARY_ONLY_BAMFILE ="data/dmgoth101_genome_alignments/bam/FC30.against_dmgoth.filtered_max_AS.primary_only.bam"
FC29_DMGOTH_MAX_AS_PRIMARY_ONLY_BAMFILE ="data/dmgoth101_genome_alignments/bam/FC29.against_dmgoth.filtered_max_AS.primary_only.bam"
GENE_ANNOTATIONS_FILE = "data/dmgoth101_genome_alignments/annotations/Dm_Goth_10-1.dmel6.23LiftOff.sorted.gff"
# TE_CLASSIFICATION_FILE= "/data2/eric/TE_LR_RNAseq/TE_hierarchy.tsv"
TE_CLASSIFICATION_FILE_V3= "data/dmgoth101_genome_alignments/annotations/TE_classification.from_RM_output_v3.csv"
TE_CLASSIFICATION_FILE_V2= "data/dmgoth101_genome_alignments/annotations/TE_classification.from_RM_output_v2.csv"
# BLAST_TE_ANNOTATIONS_FILE = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/annotations/Dm_Goth_10-1_insertions_vsTEdb.gtf"
RM_TE_ANNOTATION_FILE_V3 = "data/dmgoth101_genome_alignments/annotations/dmgoth101.onecode.v3.gtf"
RM_TE_ANNOTATION_FILE_V2 = "data/dmgoth101_genome_alignments/annotations/dmgoth101.onecode.v2.gtf"

# INSERTION_TABLE = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/insertion_table.tsv"

# GLOBAL PARAMETERS

MIN_SUB_COVERAGE = 0.1 # Threshold used to filter the features (gene or TE) mapped by a read = minimal subject coverage (nb of aligned bases / total nb of feature's bases)


In [13]:
class Gene:
	def __init__(self, chrom, start, end, gene_id):
		self.chrom = chrom
		self.start = start
		self.end = end
		self.gene_id = gene_id
		self.exons = set()
		self.mrna = set()

	def __len__(self):
		return self.end - self.start

	def __repr__(self):
		return self.gene_id


def parse_gene_annotation_line(line):
    sline = line.strip().split("\t")
    chrom = sline[0]
    start = int(sline[3])
    end = int(sline[4])
    gene_id = sline[-1].split(";")[0].split('"')[1]
    return Gene(chrom, start, end, gene_id)


def is_overlapped(start1, end1, start2, end2):
    return end1 >= start2 and end2 >= start1


def get_gene_dict(gene_annotation):
	gene_dict = {}
	with open(gene_annotation, 'r') as gene_annot:
		for line in gene_annot:
			sline = line.strip().split('\t')
			gene_id = sline[-1].split(";")[0].split('"')[1]
			if sline[2] == "gene":
				chrom = sline[0]
				new_gene = parse_gene_annotation_line(line)
				if chrom in gene_dict:
					gene_dict[chrom][new_gene.gene_id] = new_gene
				else:
					gene_dict[chrom] = {new_gene.gene_id: new_gene}
			elif sline[2] == "exon":
				gene_dict[chrom][gene_id].exons.add(
					(int(sline[3]), int(sline[4])))
			elif sline[2] == "mRNA":
				gene_dict[chrom][gene_id].mrna.add(
					(int(sline[3]), int(sline[4])))
	return gene_dict


def read_TE_classification_file(TE_classification_file):
	"""From a tsv table with 3 columns : Family, Class, Superfamily,
	return a pandas dataframe.

	Args:
		TE_classification_file (str): path to the TE classification table (tsv format)
	"""
	with open(TE_classification_file, 'r') as input:
		input.readline()
		families = list()
		superfamilies = list()
		subclasses = list()
		for line in input:
			if len(line.split("\t")) == 3:
				line = line.strip()
				family, subclass, superfamily = line.split("\t")
				families.append(family)
				superfamilies.append(superfamily)
				subclasses.append(subclass)
	TE_classification_df = pd.DataFrame(list(zip(subclasses, superfamilies, families)), columns=[
	                                    'Subclass', 'Superfamily', 'Family'])
	return TE_classification_df


class TE_feature:
	def __init__(self, chrom, start, end, gene_id, insertion_id):
		self.chrom = chrom
		self.start = start
		self.end = end
		self.insertion_id = insertion_id
		self.count = 0
		# self.family = gene_id_to_family_name(gene_id)
		self.counted_reads = set()
		self.chimeric_reads = set()

	def __len__(self):
		return self.end - self.start

	def __repr__(self):
		return self.insertion_id

	def is_valid(self, bam_chromosomes):
		return len(self) > 150 and self.chrom in bam_chromosomes


def build_TE(line):
    sline = line.strip().split("\t")
    chrom = sline[0]
    start = int(sline[3])
    end = int(sline[4])
    gene_id = sline[-1].split(";")[0].split('"')[1]
    insertion_id = sline[-1].strip().split('transcript_id "')[-1][:-2]

    return TE_feature(chrom, start, end, gene_id, insertion_id)


def filter_relevant_TE_feature(bamfile, TE_annotation_file, min_TE_size=150):
	"""Generate list of TE objects that will next be counted.
	TE are filtered : we discard those which are on chromosome absent from the bamfile
	and those with length (in number of base) below a certain threshold.

	Args:
		bamfile (str): path to the alignment file
		TE_annotation_file (str): path the TE annotation file (gtf format)
		min_TE_size(int): minimal number of base for a TE to be considered as valid. Default = 150
	"""
	# Enumerating chromosomes present in the bamfile
	bam_chromosomes = pysam.AlignmentFile(bamfile).references
	# Then iterating through TE_annotation_file, creating and checking TE objects
	valid_TE_list = list()
	with open(TE_annotation_file, "r") as TE_annot:
		for line in TE_annot:
			new_TE = build_TE(line)
			if new_TE.is_valid(bam_chromosomes):
				valid_TE_list.append(new_TE)
	return valid_TE_list


def regroup_TE_by_chrom(TE_feature_list):
	"""Return a dict of key:chrom and values:list of TE

	Args:
		TE_feature_list (list): flat list of TE_feature
	"""
	TE_dict = dict()
	for insertion in TE_feature_list:
		if insertion.chrom not in TE_dict:
			TE_dict[insertion.chrom] = [insertion]
		else:
			TE_dict[insertion.chrom].append(insertion)
	return TE_dict


def get_all_reads_covering_TE(TE_by_chrom, bamfile):
	"""Get a list of all reads that cover a TE.

	Args:
		TE_by_chrom (dict): dict of TE_feature ordered by chromosome
		bamfile (str): path to the alignment file
	"""
	reads_mapped_on_TE = set()
	with pysam.AlignmentFile(bamfile) as bam:
		for chrom, TE_list in tqdm(TE_by_chrom.items(), desc="Chromosome", position=0, leave=True):
			for TE in tqdm(TE_list, desc=chrom, position=0, leave=True):
				reads_fetched = list(bam.fetch( contig=TE.chrom, start=TE.start, stop=TE.end) )
				# Only keeping reads that are at least aligned once in a TE insertion :
				overlapped_reads = [read for read in reads_fetched if read.get_overlap(TE.start, TE.end)]
				reads_mapped_on_TE.update(overlapped_reads)
	return reads_mapped_on_TE


# TODO don't forget to fix comments when done
def get_overlapped_genes(read, gene_dict):
	"""Takes a read and a dictionnary of dictionnaries (chrom -> gene_id -> [start, end])
	 and returns a list of all overlapped genes.

	Args:
		read (pysam.AlignedSegment): the read we want to map
		gene_dict (dict): a dict of dict (gene start/end by gene_id by chrom)

	Returns:
		overlapped_genes_dict (dict) : dict of all genes covered by the read with their start and end position as values

	"""
	overlapped_genes = list()
	chrom = read.reference_name
	gene_list = gene_dict[chrom].values()
	overlapped_genes = [gene for gene in gene_list if is_overlapped(
	    read.reference_start, read.reference_end, gene.start, gene.end)]
	return overlapped_genes

	# overlapped_genes = list()
	# chrom = read.reference_name
	# overlapped_genes = gene_df[(gene_df.feature == "gene") & (gene_df.seqname == chrom) & (read.reference_end >= gene_df.start) & (gene_df.end >= read.reference_start)]
	# genes_positions = [(start, end) for start, end in zip(list(overlapped_genes.start), list(overlapped_genes.end))]
	# overlapped_genes_dict = dict(zip(list(overlapped_genes.gene_id), genes_positions))
	# return overlapped_genes_dict


def get_overlapped_TE(read, TE_by_chrom):
	"""Get all TE feature that are covered by a read

	Args:
		read (pysam.AlignedSegment): the read we want to map
		TE_by_chrom (dict): dict of TE_feature with key:chrom, value:list of TE
	"""
	list_of_overlapped_TE = list()
	chrom = read.reference_name
	for insertion in TE_by_chrom[chrom]:
		if read.get_overlap(insertion.start, insertion.end):  # if at least 1 base aligned in TE
			list_of_overlapped_TE.append(insertion)
	return list_of_overlapped_TE


def get_subject_coverage(alignment, insertion):
	insertion_length = insertion.end - insertion.start
	overlap_start = max(alignment.reference_start, insertion.start)
	overlap_end = min(alignment.reference_end, insertion.end)
	overlap_length = overlap_end - overlap_start
	insertion_length = insertion.end - insertion.start
	if insertion_length != 0:
		subject_coverage = overlap_length / insertion_length
	else:
		subject_coverage = 0
	return subject_coverage

def get_nb_of_non_overlapping_bases(x_start, x_end, y_start, y_end):
	start_overlap = abs(x_start - y_start)
	end_overlap = abs(x_end - y_end)
	return start_overlap + end_overlap


def choose_feature_with_less_non_overlapping_bases(read, list_of_TE, list_of_gene):
	# Return TE if TE is the best feature, else 0
	min_nob = 999999999999
	optimal_feature = None
	for TE in list_of_TE:
		nb_of_nob = get_nb_of_non_overlapping_bases(TE.start, TE.end, read.reference_start, read.reference_end)
		if nb_of_nob < min_nob:
			min_nob = nb_of_nob
			optimal_feature = TE
	for gene in list_of_gene:
		for mrna_start, mrna_end in gene.mrna:
			nb_of_nob = get_nb_of_non_overlapping_bases(mrna_start, mrna_end, read.reference_start, read.reference_end)
			if nb_of_nob < min_nob:
				return 0
		nb_of_nob = get_nb_of_non_overlapping_bases(gene.start, gene.end, read.reference_start, read.reference_end)
		if nb_of_nob < min_nob:
			return 0
	return optimal_feature


def get_gene_exonic_positions(gene_id, gene_df):
	"""Used to get exonic positions of a gene

	Args:
		gene_id (str): the id of the gene we want to look at
		gene_df (pandas.DataFrame): a df describing the gene annotation (gtf)

	Returns:
		List of Tuples: List of tuples with start and end positions of each exons of the gene
	"""
	exons=gene_df[(gene_df.gene_id == gene_id) and (gene_df.feature == "exon")]
	exonic_positions=[(start, end)
	                   for start, end in zip(list(exons.start), list(exons.end))]
	return exonic_positions

def check_if_exonic_alignment(read, gene):  # TODO : clean comments
	"""Return True if a read has at least 1 base aligned with an exon of a gene.

	Args:
		read (pysam.AlignedSegment): a pysam read
		gene (Gene):

	Returns:
		bool: True if aligned in exonic region, else false
	"""
	exonic_positions=sorted(list(gene.exons))
	for exonStart, exonEnd in exonic_positions:
		if read.reference_end >= exonStart and exonEnd >= read.reference_start:
			return True
	return False

	# gene = gene_df[gene_df.gene_id == gene_id]
	# if str(gene.seqname) != read.reference_name or read.reference_end < gene.start:
	# 	return False
	# if read.reference_end >= gene.start and gene.end >= read.reference_start :
	# 	exonic_positions = get_gene_exonic_positions(gene_id, gene_df)
	# 	for exonStart, exonEnd in exonic_positions :
	# 		if read.get_overlap(exonStart, exonEnd):
	# 			return True
	# return False

def default_filter(alignment, insertion, subcov_threshold):
    subject_coverage=get_subject_coverage(alignment, insertion)
    nb_aligned_pairs=alignment.get_overlap(insertion.start, insertion.end)
    is_ok=(subject_coverage > subcov_threshold and nb_aligned_pairs > 1)
    return is_ok

def increment_TE_count(TE_by_chrom, reads_list, gene_dict, subcov_threshold=MIN_SUB_COVERAGE):  # TODO : fix Args !
	"""Iterate through each read that covers a TE and decide if we increment (or not)
	its read counter. This decision depends on several conditions described in the filter function.
	Return an update dict of TE, with counting + several other counters
		(1) : total_nb_of_reads covering a TE
		(2) : nb_of_chimeric_reads : count the nb of chimeric read (non-linear alignment)
		(3) : the read must not be better aligned to another feature
		(4) : TODO : CO-EXPRESSED ?

	Args:
		TE_by_chrom (dict): dict of TE_feature with key:chrom, value:list of TE
		reads_list (list): list of all the reads that cover a TE locus
		gene_annotation (dict): dict of Gene_feature with key:chrom, value:list of TE
	"""
	not_covered_by_min_subcov = 0
	best_aligned_on_gene = 0
	exonic = 0
	intronic = 0
	multiple_TE = 0
	non_ambiguous = 0
	total_read = len(reads_list)
	with open('chimeric_that_pass_all_filters.v2.female.tsv', 'w') as output:
		for read in tqdm(reads_list, position=0, leave=True):
			# (1) Fetching TE insertions that are at least covered by one base.
			overlapped_TE=get_overlapped_TE(read, TE_by_chrom)

			# (2) filtering TE that are not covered by at least 10% of their length and at least 1 aligned base
			overlapped_TE=[TE for TE in overlapped_TE if default_filter(read, TE, subcov_threshold)]
			if not overlapped_TE :
				not_covered_by_min_subcov += 1
				continue

			# (3) filtering feature with the less non-overlapping bases
			overlapped_genes=get_overlapped_genes(read, gene_dict)
			overlapped_in_gene_exons=[gene for gene in overlapped_genes if check_if_exonic_alignment(read, gene)]

			chosen_feature=choose_feature_with_less_non_overlapping_bases(read, overlapped_TE, overlapped_in_gene_exons)
			if isinstance(chosen_feature, TE_feature):
				chosen_feature.count += 1
				chosen_feature.counted_reads.add(read)
				if overlapped_genes :
					if overlapped_in_gene_exons :
						exonic += 1
						# new_line = "\t".join([chosen_feature.insertion_id, read.query_name, read.reference_name, str(read.reference_start), str(read.reference_end)]) + "\n"
						chimeric = True
						for gene in overlapped_genes :
							for exon in gene.exons:
								if is_overlapped(exon[0], exon[1], chosen_feature.start, chosen_feature.end):
									chimeric = False
									break
						if chimeric :
							new_line = "\t".join([chosen_feature.insertion_id, read.query_name, read.reference_name, str(read.reference_start), str(read.reference_end)]) + "\n"
							output.write(new_line)
					else :
						intronic += 1
				else :
					if len(overlapped_TE) > 1:
						multiple_TE += 1
					else :
						non_ambiguous += 1
			else :
				best_aligned_on_gene += 1
	counters = [total_read, non_ambiguous, multiple_TE, exonic, intronic, not_covered_by_min_subcov, best_aligned_on_gene]
	return counters



def generate_counting(bamfile, TE_annotation_file, gene_annotation_file):
	"""Measure the expression of each TE by number of mapped reads.
	Return a dict with key = insertion_name and value = a list of mapped read

	Args:
		bamfile (str): path to the alignment file
		TE_annotation_file (str): path to the TE annotation file in gtf format
		gene_annotation_file (str): path to the gene annotation file in gtf format
		TE_classification_file (str): path to a tsv table with TE classification (family superfamily, subclass)
	"""
	print("Filtering valid TE...")
	gene_dict=get_gene_dict(gene_annotation_file)
	filtered_TE_features=filter_relevant_TE_feature(bamfile, TE_annotation_file)
	TE_by_chrom=regroup_TE_by_chrom(filtered_TE_features)
	print("Recovering reads covering TE...")
	reads_covering_TE=get_all_reads_covering_TE(TE_by_chrom, bamfile)
	print("Done")
	print("Incrementing TE counts...")
	counters_results=increment_TE_count(TE_by_chrom, reads_covering_TE,  gene_dict)
	print("Done")
	return [TE_by_chrom, counters_results]

def from_raw_counting_to_df(TE_dict, TE_classification_file):
	"""Convert dict of counting generated by func "generate_counting" into a more user-friendly dataframe.

	Args:
		TE_dict (dict): dictionnary of every TE insertion by chromosome with counting and all...
		TE_classification_file (str): path to a tsv table with TE classification (family superfamily, subclass)

	Returns:
		pd.DataFrame: a pandas dataframe containing informations about each TE insertions (ID, counting, reads counted...)
	"""
	TE_classification_df=pd.read_csv(TE_classification_file, sep="\t", names=[
	                                 "Subclass", "Superfamily", "Family", "Insertion"])
	flat_insertion_list=[]
	for insertion_list in TE_dict.values():
		flat_insertion_list += insertion_list
	insertion_dict=dict(zip([insertion.insertion_id for insertion in flat_insertion_list], [
	                    insertion for insertion in flat_insertion_list]))
	counting_list=[insertion.count for insertion in flat_insertion_list]
	counted_read_list=[
	    insertion.counted_reads for insertion in flat_insertion_list]
	counting_df=pd.DataFrame(list(zip(insertion_dict.keys(), counting_list, counted_read_list)),
				columns=['Insertion', 'Counting', 'Reads'])
	counting_df=TE_classification_df.merge(counting_df, on="Insertion")
	add_mean_subject_coverage_to_df(counting_df, insertion_dict)
	return counting_df

def get_insertion_mean_subject_coverage(insertion_id, counted_reads, insertion_dict):

    insertion=insertion_dict[insertion_id]
    mean_subject_coverage=0
    for read in counted_reads:
        mean_subject_coverage += get_subject_coverage(read, insertion)
    if len(counted_reads) == 0:
        return 0
    else:
        return mean_subject_coverage/len(counted_reads)

def add_mean_subject_coverage_to_df(counting_df, insertion_dict):
    counting_df["mean_subcov"]=counting_df.apply(lambda x: get_insertion_mean_subject_coverage(
        x['Insertion'], x['Reads'], insertion_dict), axis=1)
    return counting_df


In [14]:
def generate_TE_df(bamfile, TE_gtf, gene_gtf, TE_classification_file):
	# with cProfile.Profile() as pr:
	TE_by_chrom, counters_results = generate_counting(bamfile, TE_gtf, gene_gtf)
	print(counters_results)
	big_TE_df = from_raw_counting_to_df(TE_by_chrom, TE_classification_file)
	return [big_TE_df, counters_results]

# FC29_TE_df, FC29_counters = generate_TE_df(FC29_DMGOTH_MAX_AS_PRIMARY_ONLY_BAMFILE, RM_TE_ANNOTATION_FILE_V3, GENE_ANNOTATIONS_FILE, TE_CLASSIFICATION_FILE_V3)
FC30_TE_df, FC30_counters = generate_TE_df(FC30_DMGOTH_MAX_AS_PRIMARY_ONLY_BAMFILE, RM_TE_ANNOTATION_FILE_V3, GENE_ANNOTATIONS_FILE, TE_CLASSIFICATION_FILE_V3)

# FC29_TE_df, FC29_counters = generate_TE_df(FC29_DMGOTH_MAX_AS_PRIMARY_ONLY_BAMFILE, RM_TE_ANNOTATION_FILE_V2, GENE_ANNOTATIONS_FILE, TE_CLASSIFICATION_FILE_V2)
# FC30_TE_df, FC30_counters = generate_TE_df(FC30_DMGOTH_MAX_AS_PRIMARY_ONLY_BAMFILE, RM_TE_ANNOTATION_FILE_V2, GENE_ANNOTATIONS_FILE, TE_CLASSIFICATION_FILE_V2)



Filtering valid TE...
Recovering reads covering TE...


Chromosome: 100%|██████████| 6/6 [00:42<00:00,  7.11s/it]


Done
Incrementing TE counts...


100%|██████████| 7332/7332 [03:46<00:00, 32.35it/s]


Done
[7332, 489, 101, 452, 280, 1386, 4624]


In [2]:
def save_counting_df(counting_df, csv_file):
    counting_df.to_csv(csv_file, sep = '\t', index=False)

save_counting_df(FC29_TE_df, "/data2/eric/TE_LR_RNAseq/countings/FC29_counting_df.v3.tsv")
save_counting_df(FC30_TE_df, "/data2/eric/TE_LR_RNAseq/countings/FC30_counting_df.v3.tsv")

NameError: name 'FC29_TE_df' is not defined

In [29]:
# def save_counting_df(counting_df, csv_file):
#     counting_df.to_csv(csv_file, sep = '\t', index=False)

# # save_counting_df(FC29_counting_df, "FC29_counting_df.v2.tsv")
# # save_counting_df(FC30_counting_df, "FC30_counting_df.v2.tsv")

# ### SAVING COUNTING_DF WITH CHIMERIC READS EXCLUDED
# # save_counting_df(FC29_counting_df, "FC29_counting_df.v2.chimeric_reads_excluded.tsv")
# # save_counting_df(FC30_counting_df, "FC30_counting_df.v2.chimeric_reads_excluded.tsv")

# ### SAVING COUNTING_DF co_expressed
# save_counting_df(FC29_TE_df, "/data2/eric/TE_LR_RNAseq/countings/FC29_counting_df.v3.tsv")
# save_counting_df(FC30_TE_df, "/data2/eric/TE_LR_RNAseq/countings/FC30_counting_df.v3.tsv")

In [4]:
# df = expression_df.copy()
# df = df[df["Counting"] > 2]
# df["exp_diff"] = df["Expression"] - df["Expression_no_chimere"]
# fig = px.scatter(df, y="exp_diff", x="Insertion", color="Family", size="Insertion_length")
# # fig.update_traces(marker_size=10)
# fig.show()

In [3]:
# import plotly.express as px
# df = expression_df.copy()
# df = df[df["Counting"] > 2]
# df["count_diff"] = df["Counting"] - df["Counting_no_chimere"]
# fig = px.scatter(df, y="count_diff", x="Insertion", color="Family", size="Insertion_length")
# # fig.update_traces(marker_size=10)
# fig.show()

### IMPORT COUNTINGS

In [16]:
# pd.set_option('display.max_rows', 100)
# saved_FC29_counting_df_v3.sort_values(by=['Counting'], ascending=False).head(100)
# print(saved_FC29_counting_df_v3[saved_FC29_counting_df_v3['Family'] == "Copia_LTR"])

In [39]:
def get_insertion_length(insertion_name):
	start, end = insertion_name.split('$')[-2:]
	return int(end) - int(start) + 1

def get_insertion_merged_df(female_counting, male_counting):
	insertion_list = set(list(female_counting["Insertion"]) + list(male_counting["Insertion"]))
	subclass_list = []
	superfamily_list = []
	family_list = []
	male_counting_list = []
	female_counting_list = []
	insertion_length_list = []
	for insertion in insertion_list :
		if insertion in list(female_counting["Insertion"]) and insertion in list(male_counting["Insertion"]):
			insertion_df = female_counting[female_counting["Insertion"] == insertion]
			female_counting_list.append(insertion_df["Counting"].values[0])
			male_counting_list.append(male_counting[male_counting["Insertion"] == insertion]["Counting"].values[0])
			
		elif insertion in list(female_counting["Insertion"]) :
			insertion_df = female_counting[female_counting["Insertion"] == insertion]
			female_counting_list.append(insertion_df["Counting"].values[0])
			male_counting_list.append(0)
		else :
			insertion_df = male_counting[male_counting["Insertion"] == insertion]
			male_counting_list.append(insertion_df["Counting"].values[0])
			female_counting_list.append(0)
		subclass_list.append(insertion_df["Subclass"].values[0])
		superfamily_list.append(insertion_df["Superfamily"].values[0])
		family_list.append(insertion_df["Family"].values[0])
		insertion_length_list.append(get_insertion_length(insertion))

	insertion_merged_df = pd.DataFrame(list(zip(subclass_list, superfamily_list, family_list, insertion_list, insertion_length_list, female_counting_list, male_counting_list )), columns=["Subclass", "Superfamily", "Family", "Insertion", "Length", "Female_counting", "Male_counting"])

	return insertion_merged_df

insertion_merged_df = get_insertion_merged_df(saved_FC30_counting_df_v3, saved_FC29_counting_df_v3)
# print(insertion_merged_df)

In [40]:
insertion_merged_df

,Subclass,Superfamily,Family,Insertion,Length,Female_counting,Male_counting
0,LINE,I-Jockey,G4_DM,G4_DM$3R_RaGOO$2542636$2543942,1307,0,2
1,LTR,Gypsy,Gypsy9_I,Gypsy9_I$2R_RaGOO$129419$129869,451,0,0
2,LINE,I-Jockey,DOC5_DM,DOC5_DM$2R_RaGOO$1199863$1200279,417,0,0
3,LTR,Gypsy,Gypsy2-I_DM,Gypsy2-I_DM$2R_RaGOO$32165$32623,459,0,0
4,LINE,I-Jockey,DOC5_DM,DOC5_DM$2R_RaGOO$373938$374143,206,0,0
...,...,...,...,...,...,...,...
10660,LTR,Gypsy,DMLTR5,DMLTR5$3L_RaGOO$26377510$26377809,300,0,0
10661,DNA,P,PROTOP_A,PROTOP_A$3L_RaGOO$24240644$24240874,231,0,0
10662,DNA,TcMar-Tc1,TC1_DM,TC1_DM$3L_RaGOO$21112446$21113000,555,0,1
10663,LTR,Gypsy,HMSBEAGLE_I,HMSBEAGLE_I$3L_RaGOO$24883991$24885452,1462,0,0


In [14]:
### Merging the female and male dataframe by family and adding more info like nb of expressed insertion and insertion length...

def get_insertion_length(insertion_name):
	start, end = insertion_name.split('$')[-2:]
	return int(end) - int(start) + 1

def create_summary_table(counting_table):
	subclass_list = []
	superfamily_list = []
	family_list = counting_table["Family"].unique()
	min_TE_length_list = []
	max_TE_length_list = []
	counting_list = []
	nb_expressed_insertion_list = []
	nb_insertion_list = []
	most_expressed_insertion_list = []
	for family in family_list :
		family_df = counting_table[counting_table["Family"] == family]
		subclass_list.append(family_df["Subclass"].values[0])
		superfamily_list.append(family_df["Superfamily"].values[0])
		min_TE_length = 9999999
		max_TE_length = 0
		for insertion_name in family_df["Insertion"]:
			insertion_length = get_insertion_length(insertion_name)
			if insertion_length < min_TE_length :
				min_TE_length = insertion_length
			if insertion_length > max_TE_length :
				max_TE_length = insertion_length
		min_TE_length_list.append(min_TE_length)
		max_TE_length_list.append(max_TE_length)
		nb_insertion_list.append(len(family_df["Insertion"]))
		counting_list.append(family_df["Counting"].sum())
		nb_expressed_insertion_list.append(len(family_df[family_df["Counting"] > 0]))
		most_expressed_insertion_list.append(family_df[family_df["Counting"] == family_df["Counting"].max()]["Insertion"].values[0])
    
	summary_df = pd.DataFrame(list(zip(subclass_list, superfamily_list, family_list, min_TE_length_list, max_TE_length_list, counting_list, nb_insertion_list, nb_expressed_insertion_list, most_expressed_insertion_list)), columns=["Subclass", "Superfamily", "Family", "Min_TE_length","Max_TE_length", "Counts", "nb_of_insertion", "nb_of_expressed_insertion", "Most_expressed_insertion"])
	return summary_df


def merge_female_and_male_df(female_counting, male_counting):
	female_df = create_summary_table(female_counting)
	male_df = create_summary_table(male_counting)
	hierarchy_cols = female_df[["Subclass", "Superfamily", "Family", "nb_of_insertion", "Min_TE_length","Max_TE_length"]]
	hierarchy_df = hierarchy_cols.copy()
	full_summary_df = hierarchy_df.merge(female_df.drop(columns=["Subclass", "Superfamily", "Min_TE_length","Max_TE_length", "nb_of_insertion"]), on='Family').merge(male_df.drop(columns=["Subclass", "Superfamily", "Min_TE_length","Max_TE_length", "nb_of_insertion"]), on='Family')
	full_summary_df.columns = ["Subclass", "Superfamily", "Family", "nb_of_insertion", "Min_TE_length", "Max_TE_length", "Female_Counts", "Female_nb_of_expressed_insertion", "Female_most_expressed_insertion", "Male_Counts", "Male_nb_of_expressed_insertion", "Male_most_expressed_insertion"]

	return full_summary_df

merged_full_summary_df = merge_female_and_male_df(saved_FC30_counting_df_v3, saved_FC29_counting_df_v3)
# print(merged_full_summary_df)
# save_counting_df(merged_full_summary_df, "merged_full_summary.v2.tsv")

# def create_table_from_counting(female_counting, male_counting, hierarchy):
#     male_counting = male_counting.drop(columns=["Min_TE_length", "Max_TE_length", "nb_of_insertion"])
#     TE_data_table = hierarchy.merge(female_counting, on='Family').merge(male_counting, on='Family')
#     TE_data_table.columns = ["Subclass", "Superfamily", "Family", "Min_TE_length", "Max_TE_length", "Female_Counts", "Nb_of_insertion", "Female_nb_of_expressed_insertion", "Female_most_expressed_insertion", "Male_Counts", "Male_nb_of_expressed_insertion", "Male_most_expressed_insertion"]
#     TE_data_table = TE_data_table[["Subclass", "Superfamily", "Family", "Nb_of_insertion", "Min_TE_length", "Max_TE_length", "Female_Counts", "Female_nb_of_expressed_insertion", "Female_most_expressed_insertion", "Male_Counts", "Male_nb_of_expressed_insertion", "Male_most_expressed_insertion"]]
#     return TE_data_table

In [15]:
merged_full_summary_df

,Subclass,Superfamily,Family,nb_of_insertion,Min_TE_length,Max_TE_length,Female_Counts,Female_nb_of_expressed_insertion,Female_most_expressed_insertion,Male_Counts,Male_nb_of_expressed_insertion,Male_most_expressed_insertion
0,LINE,I,IVK_DM,39,196,7386,3,2,IVK_DM$3L_RaGOO$19656431$19661800,3,2,IVK_DM$3L_RaGOO$19656431$19661800
1,LTR,Gypsy,DM297_I,63,171,10825,5,4,DM297_I$2R_RaGOO$2216579$2220068,4,3,DM297_I$2R_RaGOO$16168$20202
2,LINE,I-Jockey,TAHRE,20,164,17305,8,5,TAHRE$2R_RaGOO$3700966$3703361,590,12,TAHRE$2R_RaGOO$1145909$1151824
3,LINE,I-Jockey,TART-A,13,169,8596,9,3,TART-A$3L_RaGOO$24811306$24813320,196,9,TART-A$X_RaGOO$3$3271
4,LTR,Gypsy,MDG1_LTR,27,170,7346,4,4,MDG1_LTR$2L_RaGOO$1126248$1133534,5,3,MDG1_LTR$3L_RaGOO$22730419$22737426
...,...,...,...,...,...,...,...,...,...,...,...,...
192,LTR,Gypsy,Chimpo_I,5,1166,4154,1,1,Chimpo_I$4_RaGOO$776018$780171,0,0,Chimpo_I$3L_RaGOO$24683105$24684455
193,LTR,Unknown,FUSHI_DM,1,154,154,0,0,FUSHI_DM$3R_RaGOO$6278611$6278764,0,0,FUSHI_DM$3R_RaGOO$6278611$6278764
194,LTR,Unknown,ARS406_DM,4,194,256,0,0,ARS406_DM$X_RaGOO$9619999$9620210,0,0,ARS406_DM$X_RaGOO$9619999$9620210
195,DNA,RC,Helitron1_DM,1,564,564,0,0,Helitron1_DM$X_RaGOO$21992722$21993285,0,0,Helitron1_DM$X_RaGOO$21992722$21993285


In [ ]:
def from_counting_to_df(counting_dict):
    family_list = []
    min_TE_length_list = []
    max_TE_length_list = []
    counting_list = []
    nb_expressed_insertion_list = []
    nb_insertion_list = []
    most_expressed_insertion_list = []
    for family, insertion_list in counting_dict.items():
        family_list.append(family)
        nb_insertion_list.append(len(insertion_list))
        nb_expressed_insertion = 0
        min_TE_length = 9999999999
        max_TE_length = 0
        family_counts = 0
        most_expressed_insertion = None
        max_count = 0
        for insertion in insertion_list :
            insertion_length = insertion.end - insertion.start
            min_TE_length = min(min_TE_length, insertion_length)
            max_TE_length = max(max_TE_length, insertion_length)
            family_counts += insertion.counts
            max_count = max(max_count,insertion.counts)
            if insertion.counts >= max_count :
                most_expressed_insertion = insertion.id
            if insertion.counts > 0 :
                nb_expressed_insertion += 1
        nb_expressed_insertion_list.append(nb_expressed_insertion)
        min_TE_length_list.append(min_TE_length)
        max_TE_length_list.append(max_TE_length)
        counting_list.append(family_counts)
        most_expressed_insertion_list.append(most_expressed_insertion)
        
    counting_df = pd.DataFrame(list(zip(family_list, min_TE_length_list, max_TE_length_list, counting_list, nb_insertion_list, nb_expressed_insertion_list, most_expressed_insertion_list)), columns=["Family", "Min_TE_length","Max_TE_length", "Counts", "nb_of_insertion", "nb_of_expressed_insertion", "Most_expressed_insertion"])
    return counting_df

In [37]:
### Export reads mapped on the 3 expressed insertions of POGO.

#### Get ID of reads mapped on POGO

# insertion = "POGO_3L_RaGOO_9733928_9735150"
# insertion = "POGO_X_RaGOO_21863530_21864880"
# insertion = "POGO_2R_RaGOO_7201268_7202754"

# mapped_reads = saved_FC30_counting_df[saved_FC30_counting_df["Insertion"] == insertion]["Reads"]
# mapped_reads = mapped_reads.iloc[0].split("'")
# reads_IDs = []
# for i, j in enumerate(mapped_reads):
# 	if i % 2 :
# 		reads_IDs.append(j)
# test =  pysam.AlignmentFile(FC30_DMGOTH_MAX_AS_BAMFILE, "r")
# read_it = test.fetch(contig = "2R_RaGOO", start = 7201268, end = 7202754)
# with open(insertion + ".mapped_read.fasta", 'w') as output:
# 	for read in read_it:
# 		if read.query_name in reads_IDs:
# 			if read.query_sequence != None:
# 				new_line = "> " + read.query_name + "\n" + read.query_sequence + "\n"
# 				output.write(new_line)


